In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

In [2]:
Xtr = pd.read_csv('Xtr_2.csv')
Xte = pd.read_csv('Xte_2.csv')
Ytr = pd.read_csv('Ytr.csv')


In [3]:
# Creating a new feature named Ratio
merged_data = Xtr.merge(Ytr, on='ID')
merged_data['Ratio'] = merged_data['Sale_Amount'] / merged_data['Assessed_Value']
merged_data['Ratio'] = merged_data['Ratio'].replace(0, np.nan)
result = merged_data.groupby(['List_Year', 'Town'])[['Ratio']].mean().reset_index()
result['Ratio'] = result['Ratio'].replace([np.inf, -np.inf], np.nan)
label_encoder = LabelEncoder()
result['Town'] = label_encoder.fit_transform(result['Town'])
data = result[['List_Year', 'Town', 'Ratio']]

# Imputation missing Sales_Ratio by using KNNImputer
# Reason: When creating Ratio, there should be missing values since we have missing Assessed_Value.
imputer = KNNImputer(n_neighbors=30)
imputed_data = imputer.fit_transform(data)
result['Sales_Ratio'] = imputed_data[:, 2]
Town = merged_data.groupby(['List_Year', 'Town'])[['Ratio']].mean().reset_index()
result['Town'] = Town['Town']
Xtr = Xtr.merge(result[['List_Year', 'Town', 'Ratio']], on=['List_Year', 'Town'], how='left')
mean_sales_ratio = Xtr.groupby(['List_Year', 'Town'])[['Ratio']].mean().reset_index()
Xte = Xte.merge(mean_sales_ratio, on=['List_Year', 'Town'], how='left')

In [4]:
# Filling up Ratio if there are still missing
mean_fillna = Xtr.groupby(['Town'])['Ratio'].transform('median')
Xtr['Ratio'].fillna(mean_fillna, inplace=True)

mean_fillna = Xte.groupby(['Town'])['Ratio'].transform('median')
Xte['Ratio'].fillna(mean_fillna, inplace=True)

In [5]:
Xtr4 = Xtr[["List_Year", "Assessed_Value", "Town", 'Ratio']]
Xte4 = Xte[["List_Year", "Assessed_Value", "Town", 'Ratio']]

# Calculate feature1Btr
towns = Xtr4["Town"].unique()
feature1Btr = np.empty(len(Xtr4))
for town in towns:
    mask = Xtr4["Town"] == town
    mu = Ytr.loc[mask, "Sale_Amount"].mean()
    feature1Btr[mask] = mu

# Calculate feature1Bte
feature1Bte = np.zeros(len(Xte4))
for town in towns:
    mask = Xtr4["Town"] == town
    mu = Ytr.loc[mask, "Sale_Amount"].mean()
    mask = Xte4["Town"] == town
    feature1Bte[mask] = mu

# Add F1B feature
Xtr4["F1B"] = feature1Btr
Xte4["F1B"] = feature1Bte

# Drop the "Town" column
Xtr4.drop("Town", axis=1, inplace=True)
Xte4.drop("Town", axis=1, inplace=True)

In [6]:
Xtr4.to_csv('Xtr_final.csv', index=False)
Xte4.to_csv('Xte_final.csv', index=False)